In [1]:
#!conda install -y geopandas folium

In [2]:
#!pip install osmnx

In [3]:
import geopandas as gpd
import osmnx as ox
import folium
import pandas as pd

In [44]:
world_low_res = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [4]:
world = gpd.read_file('boundaries/boundaries.shp')

In [43]:
world

,OBJECTID,CNTRY_NAME,geometry
0,1,Aruba,"POLYGON ((-69.88223 12.41111, -69.94695 12.436..."
1,2,Antigua and Barbuda,"MULTIPOLYGON (((-61.73889 17.54055, -61.75195 ..."
2,3,Afghanistan,"POLYGON ((61.27656 35.60725, 61.29638 35.62853..."
3,4,Algeria,"POLYGON ((-5.15213 30.18047, -5.13917 30.19236..."
4,5,Azerbaijan,"MULTIPOLYGON (((45.02583 41.03055, 45.00999 41..."
...,...,...,...
246,247,Western Samoa,"MULTIPOLYGON (((-172.59650 -13.50911, -172.551..."
247,248,Swaziland,"POLYGON ((30.89944 -26.77195, 30.88028 -26.793..."
248,249,Yemen,"MULTIPOLYGON (((48.68639 14.03750, 48.61000 14..."
249,250,Zambia,"POLYGON ((30.21302 -14.98172, 30.21917 -15.096..."


In [5]:
uk = world[world.CNTRY_NAME == 'United Kingdom']

In [6]:
uk_poly = uk.geometry.iloc[0]

In [7]:
# Download all leisure footprints in our focus area
leisure = ox.footprints.footprints_from_polygon(uk_poly, footprint_type='leisure')

In [8]:
pitches = leisure[(leisure.leisure=='pitch') | (leisure.leisure=='stadium')]

In [9]:
soccer = pitches[pitches.sport == 'soccer']

In [10]:
pd.options.display.max_columns = 300

In [42]:
soccer.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 17856 entries, 56743474 to 11142325
Columns: 1094 entries, nodes to loc_name:en
dtypes: geometry(1), object(1093)
memory usage: 149.8+ MB


In [11]:
soccer.head()

,nodes,area,leisure,note,geometry,name,sport,dogs,surface,operator,wikidata,addr:city,addr:country,addr:county,addr:hamlet,addr:postcode,addr:street,addr:village,tourism,landuse,source,addr:housename,building,FIXME,bicycle,lit,natural,water,waterway,seamark:small_craft_facility:category,seamark:type,fixme,website,url,access,wikipedia,aprox,source:shape,contact:website,phone,wheelchair,opening_hours,description,golf:course,golf:par,old_name,designation,barrier,fence_type,note:barrier,smoking,source:id,survey:date,name:de,sport_1,sport_2,disused,highway,service,landcover,man_made,layer,dog,beauty,self_service,shop,salt,garden:style,lanes,abandoned,fhrs:id,outdoor_seating,fee,private,garden:type,area:highway,location,swimming_pool,rugby,car,foot,route,playground,alt_name,loc_name,created_by,contact:phone,contact:facebook,covered,email,leaf_cycle,leaf_type,addr:housenumber,height,abandoned:leisure,building:levels,payment:bitcoin,workshop,addr:place,addr:unit,brand,contact:email,information,toilets:wheelchair,gaelic_games:football,gaelic_games:hurling,roof:material,roof:shape,internet_access,power_supply,wall,length,payment:cash,payment:credit_cards,payment:mastercard,payment:visa,golf,amenity,old_operator,historic,colour,fitness_station,pitch:count,beer_garden,heating,weather_protection,craft,hoops,playground:theme,source:name,source:addr,opening_hours:url,geograph:id,golf_course,survey,roof,vehicle,heritage,club,seamark:information,boundary,name:gd,short_name,size,roof:colour,incline,fax,name:en,maxweight,source:location,...,previously,building:parts,contact:strava,multipolygon,not:designation,source:outine,mtb,drinking_water,fountain,Source,park_ride,drink:beer,heated,pets,addr:source,not:source,operator:behalf_of,waste_disposal,TODO,woodland_management,urk,owner:charity_number,commemorates,construction_date,protection,source:alignment,baseball,ref:os,miniature_golf:windmill,source:city,golf:course:tee_names,golf:length:ladies,golf:length:white,golf:length_unit,golf:par:ladies,golf:stroke_index,golf:stroke_index:ladies,golf:length:yellow,ref:charity,ref:historicplanningapp,wikimedia_commons_1,wikimedia_commons_2,he:inscription_date,ref:he,source:indoor_play,postcode,image:pano,url:bexhillhistorytrail,ref:planningapp,theatre:type,aerialway,builder,building:architecture,ref:thekeep,opening_hours:children_swimming,tower,website:operator,start_year,phone:urgent,strapline,addr:interpolation,former_amenity,fireplace,climbing:deepwater,climbing:ice,climbing:mixed,climbing:multipitch,climbing:trad,source:boundary,paved,old:fhrs:id,caravans,tents,bicycle:conditional,maxweight:conditional,source:phone,square,amenity_2,name:pitch,contact:snapchat,harbour:scrubbing_berth,fee:description,smoothness,shower,addr:townland,golf:course:holes,fence:type,year_built,country,day_off,day_on,gaelic_games:handball,addr:postal_district,alt_name:ga,contact:youtube,archery,shade,payment:notes,payment:visa_debit,recycling:glass,golf:course:par,common,gaelic_games:camogie,name_alt,gaelic_games:rounders,backrest,year_of_construction,miniature_golf,island,official_name:en,members,name:mk,short_name:en,short_name:gd,drive_through,zoo,FEATCODE,ID,protected_area,civilization,role,site,STAG_ADMIN:access,STAG_ADMIN:close_summer,STAG_ADMIN:close_winter,STAG_ADMIN:open_daily,STAG_BASE:landuse,STAG_LINETYPE:edge,STAG_USE:leisure,REFERENCE,Status,name:local,whc:inscription_date,source:ref,golf:couse,access:foot,studio,alt_name_2,athletics:race_walking,athletics:sprint,athletics:steeplechase,protected,opening_hours:footgolf,opening_hours:golf,golf:length:blue,golf:length:red,golf:par:red,golf:stroke_index:red,embankment,loc_name:en
56743474,"[710214765, 710214767, 710214770, 710214771, 7...",NaN,pitch,NaN,"POLYGON ((-5.19194 50.21197, -5.19069 50.21173...",NaN,soccer,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,bing,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

In [16]:
soccer.loc[:,['geometry', 'leisure', 'note', 'name', 'surface']].to_file("osm_soccer_pitches.geojson", driver='GeoJSON')

In [40]:
soccer_x = [point.x for point in soccer.geometry.centroid]
soccer_y = [point.y for point in soccer.geometry.centroid]
soccer_list = zip(soccer_y, soccer_x)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  


In [12]:
tileset_ID_str = "satellite-v9"
mapbox_access_token = "pk.eyJ1Ijoic2VieHdvbGYiLCJhIjoiY2pvbmVlbHVvMHV5cjNrcnl3ZnU0cGU3eSJ9.jP3kUXiKwKFXwQyUgutldQ"
tilesize_pixels = '512'
mapbox_tile_URL = f"https://api.mapbox.com/styles/v1/mapbox/{tileset_ID_str}/tiles/{tilesize_pixels}/{{z}}/{{x}}/{{y}}@2x?access_token={mapbox_access_token}"

In [41]:
m = folium.Map(
        location=[53, 0],
        tiles=None,
        zoom_start=8,
        control_scale = True)

folium.TileLayer('openstreetmap', name = 'OpenStreetMap').add_to(m)
folium.TileLayer(mapbox_tile_URL, attr = 'Mapbox', name = 'Satellite', overlay = False).add_to(m)

folium.GeoJson(soccer, name = 'Football Pitches', 
        style_function= lambda x: {"weight": 2, 'color': 'black', 'fillColor': 'blue', 'fillOpacity':0.8},
        highlight_function=lambda x: {'weight':4, 'color':'black'},
        smooth_factor=1,
        show = True,
        tooltip=folium.features.GeoJsonTooltip(fields = ['name', 'note', 'surface' , 'source',],
                                                  aliases=['Name', 'Note' , 'Surface', 'Source'], 
                                                  labels=True, 
                                                  sticky=True,
                                                  localize=True
                                                 )).add_to(m)

for lat, lon in soccer_list:
    folium.CircleMarker(location = [lat, lon], 
                        fill_color = 'green',
                        radius = 5,
                        color='g',
                        fill = True,
                        fill_opacity = 0.7).add_to(m)

folium.LayerControl().add_to(m)
    
m.save('./osm_soccer_pitches.html')

In [39]:
m = folium.Map(
        location=[53, 0],
        tiles=None,
        zoom_start=5,
        control_scale = True)

folium.TileLayer('openstreetmap', name = 'Map').add_to(m)
folium.TileLayer(mapbox_tile_URL, attr = 'Mapbox', name = 'Satellite', overlay = False).add_to(m)

for lat, lon in soccer_list:
    folium.CircleMarker(location = [lat, lon], 
                        fill_color = 'green',
                        radius = 5,
                        color='g',
                        fill = True,
                        fill_opacity = 0.7).add_to(m)
    

folium.LayerControl().add_to(m)
    
m.save('./osm_soccer_pitches_centroids.html')

# Europe

In [45]:
world_low_res

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,53950935,Africa,Tanzania,TZA,150600.0,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253,Africa,W. Sahara,ESH,906.5,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,35623680,North America,Canada,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."
...,...,...,...,...,...,...
172,7111024,Europe,Serbia,SRB,101800.0,"POLYGON ((18.82982 45.90887, 18.82984 45.90888..."
173,642550,Europe,Montenegro,MNE,10610.0,"POLYGON ((20.07070 42.58863, 19.80161 42.50009..."
174,1895250,Europe,Kosovo,-99,18490.0,"POLYGON ((20.59025 41.85541, 20.52295 42.21787..."
175,1218208,North America,Trinidad and Tobago,TTO,43570.0,"POLYGON ((-61.68000 10.76000, -61.10500 10.890..."


In [47]:
europe = world_low_res[world_low_res.continent == 'Europe']

In [48]:
europe_poly = europe.geometry.iloc[0]

In [ ]:
# Download all leisure footprints in our focus area
leisure_europe = ox.footprints.footprints_from_polygon(europe_poly, footprint_type='leisure')

In [ ]:
pitches_europe = leisure_europe[(leisure_europe.leisure=='pitch') | (leisure_europe.leisure=='stadium')]

In [ ]:
soccer_europe = pitches_europe[pitches_europe.sport == 'soccer']

In [ ]:
soccer_europe.loc[:,['geometry', 'leisure', 'note', 'name', 'surface']].to_file("osm_soccer_pitches_europe.geojson", driver='GeoJSON')

In [ ]:
soccer_europe_x = [point.x for point in soccer_europe.geometry.centroid]
soccer_europe_y = [point.y for point in soccer_europe.geometry.centroid]
soccer_list_europe = zip(soccer_europe_y, soccer_europe_x)

In [ ]:
m = folium.Map(
        location=[50, 9],
        tiles=None,
        zoom_start=5,
        control_scale = True)

folium.TileLayer('openstreetmap', name = 'Map').add_to(m)
folium.TileLayer(mapbox_tile_URL, attr = 'Mapbox', name = 'Satellite', overlay = False).add_to(m)

for lat, lon in soccer_list_europe:
    folium.CircleMarker(location = [lat, lon], 
                        fill_color = 'green',
                        radius = 5,
                        color='g',
                        fill = True,
                        fill_opacity = 0.7).add_to(m)
    

folium.LayerControl().add_to(m)
    
m.save('./osm_soccer_pitches_centroids_europe.html')